## Téléchargement du corpus Wikipédia  

In [2]:
! wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-meta-current.xml.bz2

apex	  discord     michel   transformers    wikiextractor
Datasets  hardcopy.0  Outputs  Untitled.ipynb


Nous ouvrons l'archite avec l'option k afin de garder le fichier d'origine, Vous pouvez ajouter l'option --quiet pour supprimer 

Dans le dossier décompressé, nous allons trouver d'autres dossiers qui chacun contiennet plusieurs fichiers que nous devrons parcourir afin de populer notre base de donées.

In [4]:
! bzip2 -dk /store/dyfar/enwiki-latest-pages-meta-current.xml.bz2

In [ ]:
! pip install wikiextractor
! mkdir /store/dyfar/wikipedia_dump
! python3 -m wikiextractor.WikiExtractor /store/dyfar/enwiki-latest-pages-meta-current.xml -o /store/dyfar/wikipedia_dump --json

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/Python-3.7.3/bin/python3.7 -m pip install --upgrade pip' command.
/usagers/dyfar/.local/lib/python3.7/site-packages/wikiextractor/WikiExtractor.py:2450: DeprecationWarning: Flags not at the start of the expression '\\[(((?i)bitcoin:|ftp' (truncated)
  re.S | re.U)
/usagers/dyfar/.local/lib/python3.7/site-packages/wikiextractor/WikiExtractor.py:2457: DeprecationWarning: Flags not at the start of the expression '^(http://|https://)(' (truncated)
  re.X | re.S | re.U)
INFO: Loaded 0 templates in 0.0s
INFO: Starting page extraction from /store/dyfar/enwiki-latest-pages-meta-current.xml.
INFO: Using 19 extract processes.
INFO: 12	Anarchism
INFO: 25	Autism
INFO: 39	Albedo
INFO: 290	A
INFO: 305	Achilles
INFO: 303	Alabama
INFO: 307	Abraham Lincoln
INFO: 309	An American in Paris
INFO: 308	Aristotle
INFO: 316	Academy Award for Best Production Design
INFO: 324	Academy Aw

In [24]:
ls /store/dyfar/wikipedia_dump

AA/  AJ/  AS/  BB/  BK/  BT/  CC/  CL/  CU/  DD/  DM/  DV/  EE/  EN/  EW/  FF/
AB/  AK/  AT/  BC/  BL/  BU/  CD/  CM/  CV/  DE/  DN/  DW/  EF/  EO/  EX/  FG/
AC/  AL/  AU/  BD/  BM/  BV/  CE/  CN/  CW/  DF/  DO/  DX/  EG/  EP/  EY/  FH/
AD/  AM/  AV/  BE/  BN/  BW/  CF/  CO/  CX/  DG/  DP/  DY/  EH/  EQ/  EZ/  FI/
AE/  AN/  AW/  BF/  BO/  BX/  CG/  CP/  CY/  DH/  DQ/  DZ/  EI/  ER/  FA/  FJ/
AF/  AO/  AX/  BG/  BP/  BY/  CH/  CQ/  CZ/  DI/  DR/  EA/  EJ/  ES/  FB/  FK/
AG/  AP/  AY/  BH/  BQ/  BZ/  CI/  CR/  DA/  DJ/  DS/  EB/  EK/  ET/  FC/
AH/  AQ/  AZ/  BI/  BR/  CA/  CJ/  CS/  DB/  DK/  DT/  EC/  EL/  EU/  FD/
AI/  AR/  BA/  BJ/  BS/  CB/  CK/  CT/  DC/  DL/  DU/  ED/  EM/  EV/  FE/


## Peuplement de la basse de données

In [1]:
import argparse
import sqlite3
import json
import os
import logging
import importlib.util
import uuid
import regex
from tqdm.auto  import tqdm
from multiprocessing import Pool as ProcessPool

logger = logging.getLogger()
logger.setLevel(logging.INFO)
fmt = logging.Formatter('%(asctime)s: [ %(message)s ]', '%m/%d/%Y %I:%M:%S %p')
console = logging.StreamHandler()
console.setFormatter(fmt)
logger.addHandler(console)

In [2]:
def iter_files(path):
    """Walk through all files located under a root path."""
    if os.path.isfile(path):
        yield path
    elif os.path.isdir(path):
        for dirpath, _, filenames in os.walk(path):
            for f in filenames:
                yield os.path.join(dirpath, f)
    else:
        raise RuntimeError('Path %s is invalid' % path)

In [3]:
def preprocess_doc(doc):
    """Preprocess a document for exemple if you want to remove stop words"""
    return doc

def split_document(doc):
    """Split a doc on each "." character"""
    return [(str(uuid.uuid4()), text) for text in doc.split('.')]

In [4]:
def get_contents(filename):
    """Parse the contents of a file. Each line is a JSON encoded document."""
    documents = []
    with open(filename, encoding='utf-8') as f:
        for line in f:
            # Parse document
            doc = json.loads(line)
            # Maybe preprocess the document with custom function
            preprocess_doc(doc)
            # Skip if it is empty or None
            if not doc:
                continue
                
                
            # Add the document
            doc = doc['text']
            docs = split_document(doc)
            documents += docs
    return documents

In [5]:
def store_contents(data_path, save_path, num_workers=1):
    """Preprocess and store a corpus of documents in sqlite.
    Args:
        data_path: Root path to directory (or directory of directories) of files
          containing json encoded documents (must have `id` and `text` fields).
        save_path: Path to output sqlite db.
        preprocess: Path to file defining a custom `preprocess` function. Takes
          in and outputs a structured doc.
        num_workers: Number of parallel processes to use when reading docs.
    """
    if os.path.isfile(save_path):
        raise RuntimeError('%s already exists! Not overwriting.' % save_path)

    logger.info('Reading into database...')
    conn = sqlite3.connect(save_path)
    c = conn.cursor()
    c.execute("CREATE TABLE documents (id PRIMARY KEY, text);")

    workers = ProcessPool(num_workers)
    files = [f for f in iter_files(data_path)]
    count = 0
    with tqdm(total=len(files)) as pbar:
        for pairs in workers.imap_unordered(get_contents, files):
            count += len(pairs)
            c.executemany("INSERT INTO documents VALUES (?,?)", pairs)
            pbar.update()
    logger.info('Read %d docs.' % count)
    logger.info('Committing...')
    conn.commit()
    conn.close()

In [7]:
store_contents("/store/dyfar/wikipedia_dump/", "/store/dyfar/doc_test.db", 19)

08/02/2020 06:51:21 PM: [ Reading into database... ]


Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Process ForkPoolWorker-10:
Process ForkPoolWorker-17:
Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Process ForkPoolWorker-13:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-12:
Process ForkPoolWorker-4:
Process ForkPoolWorker-16:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Python-3.7.3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/Python-3.7.3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
   

  File "<ipython-input-4-22d46e17f002>", line 17, in get_contents
    docs = split_document(doc)
  File "<ipython-input-4-22d46e17f002>", line 17, in get_contents
    docs = split_document(doc)
  File "/usr/local/Python-3.7.3/lib/python3.7/multiprocessing/queues.py", line 364, in put
    self._writer.send_bytes(obj)
  File "<ipython-input-4-22d46e17f002>", line 17, in get_contents
    docs = split_document(doc)
  File "<ipython-input-3-0fbf388e6d66>", line 7, in split_document
    return [(str(uuid.uuid4()), text) for text in doc.split('.')]
  File "<ipython-input-3-0fbf388e6d66>", line 7, in <listcomp>
    return [(str(uuid.uuid4()), text) for text in doc.split('.')]
KeyboardInterrupt
  File "<ipython-input-4-22d46e17f002>", line 17, in get_contents
    docs = split_document(doc)
KeyboardInterrupt
  File "<ipython-input-3-0fbf388e6d66>", line 7, in split_document
    return [(str(uuid.uuid4()), text) for text in doc.split('.')]
  File "<ipython-input-3-0fbf388e6d66>", line 7, in split

  File "/usr/local/Python-3.7.3/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
KeyboardInterrupt
  File "/usr/local/Python-3.7.3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/local/Python-3.7.3/lib/python3.7/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
KeyboardInterrupt
KeyboardInterrupt


KeyboardInterrupt: 

## Wrapper de la bd

In [8]:
import unicodedata
def normalize(text):
    """Resolve different type of unicode encodings."""
    return unicodedata.normalize('NFD', text)

In [9]:
class DocDB(object):
    """Sqlite backed document storage.
    Implements get_doc_text(doc_id).
    """

    def __init__(self, db_path=None):
        self.path = db_path
        self.connection = sqlite3.connect(self.path, check_same_thread=False)

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.close()

    def path(self):
        """Return the path to the file that backs this database."""
        return self.path

    def close(self):
        """Close the connection to the database."""
        self.connection.close()

    def get_doc_ids(self):
        """Fetch all ids of docs stored in the db."""
        cursor = self.connection.cursor()
        cursor.execute("SELECT id FROM documents")
        results = [r[0] for r in cursor.fetchall()]
        cursor.close()
        return results

    def get_doc_text(self, doc_id):
        """Fetch the raw text of the doc for 'doc_id'."""
        cursor = self.connection.cursor()
        cursor.execute(
            "SELECT text FROM documents WHERE id = ?",
            (utils.normalize(doc_id),)
        )
        result = cursor.fetchone()
        cursor.close()
        return result if result is None else result[0]

## Construction de la représentation

### Fonctions utiles pour la construction de représentation sparse

In [ ]:
from sklearn.utils import murmurhash3_32

def hash(token, vocab_size):
    """Unsigned 32 bit murmurhash for feature hashing."""
    return murmurhash3_32(token, positive=True) % vocab_size

## Recherche de documents & Ordonnancement 

## Extraction des candidats de réponse

## Sélection de la réponse